### All necessary imports

In [41]:
import urllib.request
from urllib.request import urlopen
import os
import string
import glob
import shutil
import csv
import pprint
import pandas as pd
import numpy as np
from zipfile import ZipFile
from urllib.request import urlopen
from tempfile import NamedTemporaryFile
from shutil import unpack_archive
from io import BytesIO
import logging
from boto.s3.key import Key
import ntpath
import zipfile
import sys
from os.path import basename

In [42]:
# Here I created handler, formatter, loglevel etc..
ErrorLog = 'errorLoggingP2.log'
log = logging.basicConfig(filename=ErrorLog,level=logging.DEBUG, format='%(asctime)s- %(levelname)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', filemode = 'w')#ch1 = logging.FileHandler('ErrorLog') #output the logs to a file

In [74]:
df = pd.read_csv('cik-lookup-data.txt', sep=":", names =['Company Name', 'CIK'], encoding = "ISO-8859-1" )
df['CIK']= df['CIK'].fillna(df['Company Name'].str.extract('(\d+)'))
df['Company Name'] = df['Company Name'].str.replace('\d+', '')

C:\Users\madhu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


In [77]:
df

,Company Name,CIK
!J INC,,0001438823
"#1 A LIFESAFER HOLDINGS, INC.",,0001509607
#1 ARIZONA DISCOUNT PROPERTIES LLC,,0001457512
#1 PAINTBALL CORP,,0001433777
$ LLC,,0001427189
& S MEDIA GROUP LLC,,0001447162
&TV COMMUNICATIONS INC.,,0001479357
"'MKTG, INC.'",,0000886475
'OHANA LABS INC.,,0001703629
(OURCROWD INVESTMENT IN MST) L.P.,,0001599496


In [36]:
directory = os.path.dirname(os.getcwd()) 
cur_path =  directory + '/Part_2'
newpath = '/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV'

In [33]:
argLen=len(sys.argv)
year=''
AWS_ACCESS_KEY_ID =''
AWS_SECRET_ACCESS_KEY =''

for i in range(1,len(sys.argv)):
    val=sys.argv[i]
    if val.startswith('year='):
        pos=val.index("=")
        year=val[pos+1:len(val)]
        continue
    elif val.startswith('accessKey='):
        pos=val.index("=")
        AWS_ACCESS_KEY_ID=val[pos+1:len(val)]
        continue
    elif val.startswith('secretKey='):
        pos=val.index("=")
        AWS_SECRET_ACCESS_KEY=val[pos+1:len(val)]
        continue

In [ ]:
#
if not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
    logging.warning('no access or secret key provided')
    print('No access or secret key provided')
    exit()
try:
    conn = boto.connect_s3(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
    logging.info("Successfully connected to Amazon S3")

except:
    logging.info("Invalid amazon keys")
    print("Invalid amazon keys")
    exit()

### Downloading the files in zip

In [4]:
# Unzip and downloading the files in the folder

def openUnzip(url):
    try:
        if not os.path.exists(newpath):
            os.makedirs(newpath)
            with urlopen(url) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(newpath)
        else:
            with urlopen(url) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(newpath)
    except:
        logging.warning("url:" + url + " was not correct. Check the inputted year")         

In [17]:
#year = 
if(year is None):
     logging.warning("year was left blank. Assigning the year 2005 automatically")
else:
     logging.info("year was set properly")
zipurl = 'http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/'
generalUrl = "http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"
months = ['01',"02","03","04","05","06","07","08","09","10","11","12"]
for m in range(len(months)):
    if(str(months[m]) is "01" or str(months[m]) is "02" or str(months[m]) is "03"):
        urlAppended = zipurl + str(year) +"/Qtr1/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "04" or str(months[m]) is "05" or str(months[m]) is "06"):
        urlAppended = zipurl + str(year) +"/Qtr2/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)
    if(str(months[m]) is "07" or str(months[m]) is "08" or str(months[m]) is "09"):
        urlAppended = zipurl + str(year) +"/Qtr3/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)
    if(str(months[m]) is "10" or str(months[m]) is "11" or str(months[m]) is "12"):
        urlAppended = zipurl + str(year) +"/Qtr4/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        #openUnzip(urlAppended)

### Reading the CSV in a dataframe

### Reading all CSVs and storing it in a dictionary

In [5]:
all_log_files = glob.glob(newpath+'/*.csv')
dict_csv = {}
for file_ in all_log_files:
    f_name = os.path.basename(file_)
    dict_csv[f_name] = pd.read_csv(file_,index_col=None, header=0)

logging.info("Each csv has been read into a dataframe ")

##  Data Wrangling

In [10]:
for key,value in dict_csv.items():
    df = dict_csv[key]
    
    #Dropping Cik,ip and accesion if NAN
    df.dropna(subset=['cik'])
    df.dropna(subset=['accession'])
    df.dropna(subset=['ip'])
    
    # df.isnull().sum()

    #Extension : if only extension is there, appending accession before the extension
    df['extention'] = np.where(str(df['extention']).find('.') ==0,df['extention'],df['accession']+df['extention'])
    logging.info("replaced all extensions with no extension name in front of . with accession number")
    df['extention'] = df['extention'].replace(np.nan, 'Unavailable', regex=True)
    logging.info("replaced all extensions with NAN by 'Unavailable'")

    #Zone : if NAN fill it with the max used zone value
    df['zone'] = df['zone'].fillna(df['ip'].value_counts().idxmax())
    logging.info("replaced all zones with NAN with mazimum count of ip address")
    
    #IDX : if NAN fill it with the max used idx value
    df['idx'] = df['idx'].fillna(df['idx'].value_counts().idxmax())
    logging.info("replaced all idx with NAN with value that max count")
    
    #Norefer : if NAN fill it with the max used norefer value
    df['norefer'] = df['norefer'].fillna(df['norefer'].value_counts().idxmax())
    logging.info("replaced all norefer with NAN with value that max count")

    #Noagent : if NAN fill it with the max used noagent value
    df['noagent'] = df['noagent'].fillna(df['noagent'].value_counts().idxmax())
    logging.info("replaced all noagent with NAN with value that max count")

    #Crawler : if NAN fill it with the max used crawler value
    df['crawler'] = df['crawler'].fillna(df['crawler'].value_counts().idxmax())
    logging.info("replaced all crawler with NAN with value that max count")

    #Replace size column which has NAN with the mean file size
    df['size'] = df['size'].fillna(df['size'].mean(axis=0))
    logging.info("replaced all size column with NAN with value the mean value")
    # df['date'].notnull().value
    # #sub_df.iloc[0]['A']

    #Browser : if NAN fill it with the max used browswe value
    df['browser'] = df['browser'].fillna(df['browser'].value_counts().idxmax())
    logging.info("replaced allbrowser column with NAN with the common/most used browser")

## SUMMARIES

In [ ]:
# Mean File size
#df['size_mean'] = df['size'].mean(axis=0)

In [ ]:
# Unique IPs in the given month
#df['ip_unique'] = df['ip'].nunique()

In [ ]:
#df.describe()

In [ ]:
#Unique CIK
#df['cik'].nunique()

In [35]:
#Unique Accession Number
#df['accession'].nunique()

In [ ]:
### Merging all the dataframe 






### Zipping the CSVs and the summarize

In [22]:
def zipping(directory):
 
    # initializing empty file paths list
    file_paths = []
    for root, directories, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.csv'):
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)
                
    with ZipFile('LogFiles.zip','w') as zip:
        for file in file_paths:
            #print(file)
            zip.write(file, basename(file))
 
    print('All files zipped successfully!')   
    logging.info("All files zipped successfully!")

In [23]:
zipping(newpath)

/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040101.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040201.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040301.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040401.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040501.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040601.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040701.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040801.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20040901.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20041001.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV/log20041101.csv
/Users/sonalichaudhari/Desktop/FALL2017/ADS

In [ ]:
# Uploading the zip to Amazon S3
try:   
    bucket_name = AWS_ACCESS_KEY_ID.lower()+str(st).replace(" ", "").replace("-", "").replace(":","").replace(".","")
    bucket = conn.create_bucket(bucket_name,  location=s3.connection.Location.DEFAULT)
    #zipfile = 'Problem1.zip'
    logging.info("Uploading to Amazon S3", zipfile, bucket_name)
    
    def percent_cb(complete, total):
        sys.stdout.write('.')
        sys.stdout.flush()
    
    k = Key(bucket)
    k.key = 'Assign1Problem1'
    k.set_contents_from_filename(zipfile,
        cb=percent_cb, num_cb=10)
    logging.into("Zip File successfully uploaded to S3")
except:
    logging.info("invalid amazon keys")
    print("invalid amazon keys")
    exit()